In [2]:
# For data
import numpy as np
from pandas import Series,DataFrame
import pandas as pd

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

ImportError: C extension: umpy.core.multiarray failed to import not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace' to build the C extensions first.

In [ ]:
train_filepath = './data/train.csv'
df = pd.read_csv(train_filepath)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
house_prices = df.SalePrice

In [ ]:
house_prices.head()

In [ ]:
df.count()

In [ ]:
predictor_vars_x =  ['LotArea', 'YrSold', 'MoSold', 'MSSubClass', 'MiscVal']
predictor_vars = predictor_vars_x + ['SalePrice']

In [ ]:
df_predictors = df[predictor_vars]

In [ ]:
df_predictors.describe()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
prediction_model = DecisionTreeRegressor()

In [ ]:
df_predictors_filtered = df_predictors.dropna(axis=0)

In [ ]:
df_predictors_filtered.count()

In [ ]:
prediction_model.fit(df_predictors_filtered[predictor_vars_x], df_predictors_filtered[['SalePrice']])

In [ ]:
print(df_predictors_filtered.head())
print(prediction_model.predict(df_predictors_filtered[predictor_vars_x].head()))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_vars = df_predictors_filtered[predictor_vars_x]
y_vars = df_predictors_filtered[['SalePrice']]

In [ ]:
df_predictors_filtered[predictor_vars_x].head()

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(x_vars, y_vars, random_state = 0)

In [ ]:
prediction_model.fit(train_x, train_y)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
predictions = prediction_model.predict(val_x)

In [ ]:
mean_absolute_error(predictions, val_y)

In [ ]:
def get_mae(max_leaf_nodes, predictors_train, predictors_val, targ_train, targ_val):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(predictors_train, targ_train)
    preds_val = model.predict(predictors_val)
    mae = mean_absolute_error(targ_val, preds_val)
    return(mae)

In [ ]:
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_x, val_x, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest_model = RandomForestRegressor(n_estimators=100)

In [ ]:
forest_model.fit(train_x, train_y.values.ravel())

In [ ]:
sorted(zip(map(lambda x: round(x, 4), forest_model.feature_importances_), predictor_vars_x), 
             reverse=True)

In [ ]:
predictions_train = forest_model.predict(val_x)

In [ ]:
mean_absolute_error(predictions_train, val_y)

In [ ]:
test_filepath = './data/test.csv'
csv_df = pd.read_csv(test_filepath)
df_test = csv_df[predictor_vars_x]

In [ ]:
df_test.count()

In [ ]:
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
predictions = forest_model.predict(df_test)

In [ ]:
predictions.shape

In [ ]:
df_test.head()

In [ ]:
df_test = csv_df[['Id']]

In [ ]:
df_test['SalePrice'] = predictions

In [ ]:
df_test.head()

In [ ]:
df_test.to_csv('./data/results.csv', index=False)